# Objective
The objective for this assignment is to take the in-class Multi-Layer Perceptron code we used for classification and modify it for regression.

# Approach
## MLP Class
We write an MLP class that inherits `torch.nn.Module`, the basic Neural Network module containing the required functions we'll use for Linear Regression.

The `torch.nn.Sequential` class creates a sequential container that allows us to manually call a sequence of modules. In effect, it enables us to transform the container as needed, like creating three `torch.nn.Linear` layers. The input to the first layer should be the number of features and the output of the last layer should be 1. In this case, we'll call the Sigmoid activation function to see a non-linear fit to the data. This will be graphed later in the report.

In [ ]:
import torch

class MLP(torch.nn.Module):
	def __init__(self, num_features):
		super().__init__()
		self.all_layers = torch.nn.Sequential(
            # 1st hidden layer
            torch.nn.Linear(num_features, 5),
            torch.nn.Sigmoid(),
            # 2nd hidden layer
            torch.nn.Linear(5, 2),
            torch.nn.Sigmoid(),						  
            # output layer
            torch.nn.Linear(2, 1),
        )

	def forward(self, x):
		logits = self.all_layers(x)
		return logits